In [1]:
import requests
import base64
import json
import sys
import time
import pymysql

In [2]:
def get_headers(client_id, client_secret):
    endpoint = 'https://accounts.spotify.com/api/token'
    encoded = base64.b64encode((client_id+':'+client_secret).encode('utf-8')).decode('ascii')
    headers = {'Authorization': 'Basic {auth}'.format(auth=encoded)}
    payload = {'grant_type':'client_credentials'}
    try:
        r = requests.post(endpoint, data=payload, headers=headers)
    except:
        print('인증오류가 발생되었습니다!')
        sys.exit(1)
    if r.status_code!=200:
        if r.status_code==401:
            print('Client_id와 Secret이 잘못되었습니다!')
            sys.exit(1)
        else:
            print('인증오류가 발생되었습니다!:', r.status_code)
            sys.exit(1)
    acess_token = json.loads(r.text).get('access_token')
    headers = {'Authorization': 'Bearer {auth}'.format(auth=acess_token)}
    return headers

In [3]:
def id_getter(que):
    endpoint = 'https://api.spotify.com/v1/search'
    headers = get_headers(client_id, client_secret)
    params = {'q': que,
              'type':'artist',
              'market':'KR',
              'limit':1}
    try:
        r = requests.get(endpoint, params=params, headers=headers)
    except:
        print('ID획득에 오류가 발생되었습니다!')
        sys.exit(1)
    if r.status_code!=200:
        if r.status_code==401:
            headers = get_headers(client_id, client_secret)
            r = requests.get(endpoint, params=params, headers=headers)
        elif r.status_code==429:
            retry_time = json.loads(r.headers)['Retry-After']
            time.sleep(int(retry_time) + 1)
            r = requests.get(endpoint, params=params, headers=headers)
        else:
            print('ID획득에 오류가 발생되었습니다!',r.status_code)
            sys.exit(1)
    return json.loads(r.text).get('artists').get('items')[0].get('id')

In [4]:
client_id = '77ba7a249d9940b79c687b05f518ebec'
client_secret = ''
que = 'BTS'

In [5]:
def artist_info(client_id, client_secret, que):
    try:
        artist_id = id_getter(que)
    except:
        print('ID획득에 오류가 발생되었습니다.')
        sys.exit(1)
    endpoint = 'https://api.spotify.com/v1/artists/{id}'.format(id=artist_id)
    headers = get_headers(client_id, client_secret)
    try:
        r = requests.get(endpoint, headers=headers)
    except:
        print('Artist정보 획득에 오류가 발생되었습니다.')
        sys.exit(1)
    if r.status_code!=200:
        if r.status_code==401:
            headers = get_headers(client_id, client_secret)
            r = requests.get(endpoint, params=params, headers=headers)
        elif r.status_code==429:
            retry_time = json.loads(r.headers)['Retry-After']
            time.sleep(int(retry_time) + 1)
            r = requests.get(endpoint, params=params, headers=headers)
        else:
            print('Artist정보에 오류가 발생되었습니다!',r.status_code)
            sys.exit(1)
    raw = json.loads(r.text)
    artist_dict = {}
    try:
        artist_dict['artist_id'] = raw['id']
    except:
        print('ArtistID가 없습니다!')
        sys.exit(1)
    artist_dict['name'] = raw.get('name','UNK')
    artist_dict['followers'] = raw.get('followers').get('total',0)
    artist_dict['popularity'] = raw.get('popularity',0)
    artist_dict['url'] = raw.get('external_urls').get('spotify','UNK')
    artist_dict['image_url'] = raw.get('images')[0].get('url','UNK')
    artist_genre = raw.get('genres')
    return (artist_dict, artist_genre)

In [8]:
a_info=artist_info(client_id, client_secret, que)

In [11]:
print(a_info) #artist_dict 와 artist_genre가 합쳐짐
print(a_info[0])
print(a_info[1])

({'artist_id': '3Nrfpe0tUJi4K4DXYWgMUX', 'name': 'BTS', 'followers': 42241569, 'popularity': 95, 'url': 'https://open.spotify.com/artist/3Nrfpe0tUJi4K4DXYWgMUX', 'image_url': 'https://i.scdn.co/image/ab6761610000e5eb82a5d58059f81867b871d8b6'}, ['k-pop', 'k-pop boy group'])
{'artist_id': '3Nrfpe0tUJi4K4DXYWgMUX', 'name': 'BTS', 'followers': 42241569, 'popularity': 95, 'url': 'https://open.spotify.com/artist/3Nrfpe0tUJi4K4DXYWgMUX', 'image_url': 'https://i.scdn.co/image/ab6761610000e5eb82a5d58059f81867b871d8b6'}
['k-pop', 'k-pop boy group']


In [15]:
# INSERT INTO 테이블명(열이름, ...) VALUES (값,,,) ON DUPLICATE KEY UPDATE (열이름= 값, 열이름 =값)
#위와 같은 형식으로 넣어줌
len(a_info[0])

6

In [19]:
data = a_info[0]
data.keys()
# #딕셔너리의 키들만 잡힘['artist_id', 'name', 'followers', 'popularity', 'url', 'image_url']
col = ', '.join(data.keys()) #,로 붙이기
len(data) # values(값,,,)에 넣아야할때 %s가 a_info[0]의 갯수만큼 들어와야함 
place_holders =', '.join(['%s'] *len(data))
table = 'artists'
place_holders = ', '.join(['%s']*len(data))
key_holders = ', '.join([k+'=%s' for k in data.keys()])

In [18]:
#DB에 넣는 코드(query)를 만듬
def insert_row(cursor, data, table):
    col = ', '.join(data.keys())
    place_holders = ', '.join(['%s']*len(data))
    key_holders = ', '.join([k+'=%s' for k in data.keys()])
    que = 'INSERT INTO {} ({}) VALUES ({}) ON DUPLICATE KEY UPDATE {}'.format(table, col, place_holders, key_holders)#이걸쿼리문으로 날려줘야함
    cursor.execute(que, list(data.values())* 2) #쿼리를 날려주세요 #%s가 두쌍있으니 *2

IndentationError: unexpected indent (<ipython-input-18-535018fa9b60>, line 2)

In [20]:
# %s에 들어가는건 data.values()이다.
'INSERT INTO {} ({}) VALUES ({}) ON DUPLICATE KEY UPDATE {}'.format(table, col, place_holders, key_holders)

'INSERT INTO artists (artist_id, name, followers, popularity, url, image_url) VALUES (%s, %s, %s, %s, %s, %s) ON DUPLICATE KEY UPDATE artist_id=%s, name=%s, followers=%s, popularity=%s, url=%s, image_url=%s'

In [21]:
client_id = '77ba7a249d9940b79c687b05f518ebec'
client_secret = ''

#db접속하기
# pymysql을 쓰려면 db를 열어야한다
def main():
    host = "hyundb.cqgrmpcsglic.ap-northeast-2.rds.amazonaws.com"
    port = 3306
    username = 'hyunjilee'
    database = 'production'
    pw = '!'
    #db접속 커넥션
    conn = pymysql.connect(host=host, user=username, passwd=pw, db=database,
                               port=port, use_unicode=True, charset='utf8')
    cursor = conn.cursor() #db열림
    que = 'BTS'
    artists = artist_info(client_id, client_secret, que)
    data = artists[0]
    insert_row(cursor, data, 'artists') #테이블에 업데이트
    cursor.fetchall()
    ##DB접속끝##
    conn.commit()
    cursor.close()
    cursor.execute('SELECT * FROM artists') #쿼리날리기



502